In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
# =========================================================
# IndicCorp Fetch + Augment + Label for 5 Languages
# =========================================================

import os
import csv
import random
import re
import time
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset

# ----------------------------
# CONFIGURATION
# ----------------------------
base_path = "/content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/IndicCorp_fetch_aug_label"
os.makedirs(base_path, exist_ok=True)

languages = {
    "bangla": "ben_Beng",
    "hindi": "hin_Deva",
    "malayalam": "mal_Mlym",
    "tamil": "tam_Taml",
    "telugu": "tel_Telu"
}

max_sentences = 10_000
min_length = 5
max_length = 35
augment_sample_size = 5000  # half sentences will be augmented

# ----------------------------
# Helper functions for augmentation
# ----------------------------
def char_level_augment(sentence):
    if len(sentence) < 2:
        return sentence
    aug_type = random.choice(["swap", "insert", "delete"])
    chars = list(sentence)
    if aug_type == "delete":
        del chars[random.randint(0, len(chars)-1)]
    elif aug_type == "swap" and len(chars) > 2:
        idx = random.randint(0, len(chars)-2)
        chars[idx], chars[idx+1] = chars[idx+1], chars[idx]
    elif aug_type == "insert":
        chars.insert(random.randint(0, len(chars)-1), random.choice(chars))
    return "".join(chars)

def word_level_augment(sentence):
    words = re.findall(r'\S+', sentence)
    if len(words) < 2:
        return sentence
    aug_type = random.choice(["swap", "insert", "delete"])
    if aug_type == "delete":
        del words[random.randint(0, len(words)-1)]
    elif aug_type == "swap" and len(words) > 2:
        idx = random.randint(0, len(words)-2)
        words[idx], words[idx+1] = words[idx+1], words[idx]
    elif aug_type == "insert":
        words.insert(random.randint(0, len(words)-1), random.choice(words))
    return " ".join(words)

def introduce_error(sentence):
    # randomly choose char-level or word-level
    return char_level_augment(sentence) if random.random() < 0.5 else word_level_augment(sentence)

# ----------------------------
# Main loop for all languages
# ----------------------------
for lang_name, lang_code in languages.items():
    start_time = time.time()
    print(f"\n📘 Language: {lang_name.upper()} ({lang_code})")
    print("────────────────────────────────────────")

    # Create folder for this language
    lang_folder = os.path.join(base_path, lang_name)
    os.makedirs(lang_folder, exist_ok=True)
    output_file = os.path.join(lang_folder, "Data_fetch_aug_label.csv")

    # ----------------------------
    # FETCH DATA
    # ----------------------------
    raw_lines = []
    try:
        dataset = load_dataset("ai4bharat/IndicCorpV2", "indiccorp_v2", split=lang_code, streaming=True)
        for sample in dataset:
            text = sample.get("text", "").strip()
            if text:
                raw_lines.append(text)
            if len(raw_lines) >= max_sentences * 2:  # fetch extra to allow splitting
                break
        print(f"✅ Total raw lines fetched: {len(raw_lines)}")
    except Exception as e:
        print(f"❌ Error fetching {lang_name}: {e}")
        continue

    # ----------------------------
    # SPLIT INTO SENTENCES
    # ----------------------------
    # Define sentence-ending symbols
    if lang_name in ["hindi", "bangla"]:
        split_pattern = r"[।?.]"
    else:
        split_pattern = r"[.?]"

    sentences = []
    for line in raw_lines:
        split_sents = [s.strip() for s in re.split(split_pattern, line) if s.strip()]
        sentences.extend(split_sents)
    print(f"✅ After sentence splitting: {len(sentences)}")

    # ----------------------------
    # FILTER BY LENGTH
    # ----------------------------
    filtered = [s for s in sentences if min_length <= len(s.split()) <= max_length]
    print(f"✅ After filtering (5 ≤ words ≤ 35): {len(filtered)}")

    # ----------------------------
    # SELECT FINAL 10,000 SENTENCES
    # ----------------------------
    if len(filtered) > max_sentences:
        filtered = random.sample(filtered, max_sentences)
    # Create DataFrame
    df = pd.DataFrame({"Output Sentences": filtered})
    df["Input Sentences"] = df["Output Sentences"]

    # ----------------------------
    # AUGMENT 5,000 SENTENCES
    # ----------------------------
    augment_indices = random.sample(range(len(df)), augment_sample_size)
    char_count, word_count = 0, 0
    for idx in tqdm(augment_indices, desc=f"Augmenting {lang_name}"):
        orig = df.at[idx, "Output Sentences"]
        aug = introduce_error(orig)
        df.at[idx, "Input Sentences"] = aug
        # Count type
        if aug != orig:
            if aug != orig and len(orig) == len(aug):  # simple heuristic for char-level
                char_count += 1
            else:
                word_count += 1

    print(f"✅ Final selected for dataset: {len(df)} (5,000 clean + 5,000 augmented approx.)")
    print(f"   - Char-level errors: {char_count}")
    print(f"   - Word-level errors: {word_count}")

    # ----------------------------
    # LABELING
    # ----------------------------
    df["Grammatical Error"] = df.apply(lambda row: 0 if row["Input Sentences"]==row["Output Sentences"] else 1, axis=1)

    # ----------------------------
    # SAVE CSV
    # ----------------------------
    df.to_csv(output_file, index=False, encoding="utf-8")
    elapsed = time.time() - start_time

    # ----------------------------
    # PRINT SUMMARY & FILE INFO
    # ----------------------------
    print(f"✅ Time taken: {int(elapsed//60)}m {int(elapsed%60)}s")
    print(f"💾 Saved CSV file:")
    print(f"   - {os.path.basename(output_file)}  ->  {output_file}")
    print("────────────────────────────────────────")



📘 Language: BANGLA (ben_Beng)
────────────────────────────────────────


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

✅ Total raw lines fetched: 20000
✅ After sentence splitting: 78013
✅ After filtering (5 ≤ words ≤ 35): 66835


Augmenting bangla: 100%|██████████| 5000/5000 [00:00<00:00, 26386.64it/s]


...

✅ Final selected for dataset: 10000 (5,000 clean + 5,000 augmented approx.)
   - Char-level errors: 1642
   - Word-level errors: 3358
✅ Time taken: 0m 3s
💾 Saved CSV file:
   - Data_fetch_aug_label.csv  ->  /content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/IndicCorp_fetch_aug_label/malayalam/Data_fetch_aug_label.csv
────────────────────────────────────────

📘 Language: TAMIL (tam_Taml)
────────────────────────────────────────
✅ Total raw lines fetched: 20000
✅ After sentence splitting: 75239
✅ After filtering (5 ≤ words ≤ 35): 54222


Augmenting tamil: 100%|██████████| 5000/5000 [00:00<00:00, 25227.93it/s]


✅ Final selected for dataset: 10000 (5,000 clean + 5,000 augmented approx.)
   - Char-level errors: 1654
   - Word-level errors: 3341
✅ Time taken: 0m 3s
💾 Saved CSV file:
   - Data_fetch_aug_label.csv  ->  /content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/IndicCorp_fetch_aug_label/tamil/Data_fetch_aug_label.csv
────────────────────────────────────────

📘 Language: TELUGU (tel_Telu)
────────────────────────────────────────
✅ Total raw lines fetched: 20000
✅ After sentence splitting: 80495
✅ After filtering (5 ≤ words ≤ 35): 62081


Augmenting telugu: 100%|██████████| 5000/5000 [00:00<00:00, 26640.48it/s]


✅ Final selected for dataset: 10000 (5,000 clean + 5,000 augmented approx.)
   - Char-level errors: 1638
   - Word-level errors: 3357
✅ Time taken: 0m 4s
💾 Saved CSV file:
   - Data_fetch_aug_label.csv  ->  /content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/IndicCorp_fetch_aug_label/telugu/Data_fetch_aug_label.csv
────────────────────────────────────────


In [3]:
# =========================================================
# Count Grammatical Error Labels (0 & 1) in Each Language CSV
# =========================================================

import os
import pandas as pd

# Path where the CSV files are saved for each language
base_path = "/content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/IndicCorp_fetch_aug_label"

languages = [
    "bangla", "hindi", "malayalam", "tamil", "telugu"
]

# ----------------------------
# Count the number of 0s and 1s in each language's CSV
# ----------------------------
for lang_name in languages:
    print(f"\n📘 Language: {lang_name.upper()}")

    # Path to the CSV file for the current language
    lang_folder = os.path.join(base_path, lang_name)
    input_file = os.path.join(lang_folder, "Data_fetch_aug_label.csv")

    # Check if the CSV file exists
    if os.path.exists(input_file):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(input_file)

        # Count the occurrences of 0s and 1s in the 'Grammatical Error' column
        count_0 = (df['Grammatical Error'] == 0).sum()
        count_1 = (df['Grammatical Error'] == 1).sum()

        # Print the results for this language
        print(f"✅ Grammatical Error Counts for {lang_name.capitalize()}:")
        print(f"   - 0 (No Error): {count_0}")
        print(f"   - 1 (Error): {count_1}")
    else:
        print(f"❌ No CSV found for {lang_name}")



📘 Language: BANGLA
✅ Grammatical Error Counts for Bangla:
   - 0 (No Error): 5003
   - 1 (Error): 4997

📘 Language: HINDI
✅ Grammatical Error Counts for Hindi:
   - 0 (No Error): 5004
   - 1 (Error): 4996

📘 Language: MALAYALAM
✅ Grammatical Error Counts for Malayalam:
   - 0 (No Error): 5000
   - 1 (Error): 5000

📘 Language: TAMIL
✅ Grammatical Error Counts for Tamil:
   - 0 (No Error): 5005
   - 1 (Error): 4995

📘 Language: TELUGU
✅ Grammatical Error Counts for Telugu:
   - 0 (No Error): 5005
   - 1 (Error): 4995
